**Imports**

In [1]:
import tensorflow as tf
import seaborn as sf
import matplotlib as mtb

from keras.optimizers import RMSprop
from matplotlib import pyplot as pyp
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.version)

Using TensorFlow backend.


**Check if Tensorflow is using GPU**

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

AttributeError: module 'tensorflow' has no attribute 'Session'

**Locations of the images**

The images are split in 2 directories, train and validation.
Each of those folders contain 2 directories, cats and dogs.

In [2]:
train_dir = "images/train"
validation_dir = "images/validation"

**Image generators**

Images will be turned into a 299, 299, 3 size.
Augmention will only occur in training, not validation.

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=128,
    target_size=(299, 299),
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size=20,
    target_size=(299, 299),
    class_mode='binary'
)

Found 21002 images belonging to 2 classes.
Found 3998 images belonging to 2 classes.


**Base model**

The base model chosen here is the xeception model.
The input shape is set to (299, 299, 3).
The model's weights have been trained on imagenet dataset and will not be trained anymore.
The fully connected layers will be retrained though.

In [4]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False)

**Freeze the model**

In [5]:
base_model.trainable = False

**Base model summary**

In [6]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

**Replacing fully connected layers**

The fully connected layers will be replaced with one with 1024 neuron layer followed by a 1 neuron layer for the output.

In [7]:
inputs = keras.Input(shape=(299, 299, 3))

flat1 = layers.Flatten()(base_model.output)
dense1 = layers.Dense(1024, activation='relu')(flat1)
dropout1 = layers.Dropout(0.2)(dense1)
output = layers.Dense(1, activation='sigmoid')(dropout1)

model = Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

**Full model summary**

In [8]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

**Creating callbacks**

The callbacks will only save the best model.

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

checkpoints = ModelCheckpoint(filepath='.mdl_wts.hdf5', save_best_only=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

callbacks = [earlyStopping, checkpoints, reduce_lr_loss]

**Fitting the model**


In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=100,
                    epochs=5,
                    validation_steps=50,
                    verbose=1,
                    callbacks=callbacks
)

Epoch 1/5
 18/100 [====>.........................] - ETA: 25:28 - loss: 0.1034 - acc: 0.9583

**Loading the best model**

The best model would have been stored at .mdl_wts.hdf5

In [ ]:
model = keras.models.load_model('.mdl_wts.hdf5.hdf5')